In [93]:
import matplotlib.pyplot as plt
import torch
import torchvision
from torchvision import transforms , datasets
import torch.nn as nn
import torch.nn.functional as f
import torch.optim as optim

In [94]:
train = datasets.MNIST("" , 
                       train = True  , 
                       download = True , 
                       transform = transforms.Compose([transforms.ToTensor()]))

test = datasets.MNIST("" , 
                      train = False  , 
                      download = True , 
                      transform = transforms.Compose([transforms.ToTensor()]))


In [95]:
trainset = torch.utils.data.DataLoader(train , batch_size=10)
testset = torch.utils.data.DataLoader(test , batch_size=10)

In [119]:
class network(nn.Module):
    def __init__(self , input_dim=28*28 , l1_dim=64, l2_dim=64 , l3_dim=64 , output_dim=10):
        super().__init__()
        self.l1 = nn.Linear(input_dim , l1_dim)
        self.l2 = nn.Linear(l1_dim , l2_dim)
        self.l3 = nn.Linear(l2_dim , l3_dim)
        self.l4 = nn.Linear(l3_dim , output_dim)
    def forward(self , x):
        x = f.relu(self.l1(x))
        x = f.relu(self.l2(x))
        x = f.relu(self.l3(x))
        x = f.log_softmax(self.l4(x) , dim = 1)
        return x

In [121]:
net = network()
#backtracking
optimizer = optim.Adam(net.parameters() , lr = 0.0001)
epocs = 10

for epoc in range(epocs):
    for data in trainset:
        x , y = data
        net.zero_grad()
        out = net(x.view(-1 , 28*28))
        loss = f.nll_loss(out , y)
        loss.backward()
        optimizer.step()
    print(f'loss after {epoc+1} epocs: {round(loss.item() , 6)}')

loss after 1 epocs: 0.119143
loss after 2 epocs: 0.06252
loss after 3 epocs: 0.044395
loss after 4 epocs: 0.032686
loss after 5 epocs: 0.024526
loss after 6 epocs: 0.019505
loss after 7 epocs: 0.015518
loss after 8 epocs: 0.01227
loss after 9 epocs: 0.009478
loss after 10 epocs: 0.007544


In [98]:
#calculating accuracy
correct = 0
total = 0
for data in trainset:
    x , y = data
    net.zero_grad()
    out = net(x.view(-1 , 28*28))
    for id , i in enumerate(out):
        if torch.argmax(i) == y[id]:
            correct+=1
        total+=1

print(f'Trainset Accuracy: {round(correct/total*100 , 3)}')

correct = 0
total = 0
for data in testset:
    x , y = data
    net.zero_grad()
    out = net(x.view(-1 , 28*28))
    for id , i in enumerate(out):
        if torch.argmax(i) == y[id]:
            correct+=1
        total+=1

print(f'Testset Accuracy: {round(correct/total*100 , 3)}')

Trainset Accuracy: 97.508
Testset Accuracy: 96.94
